In [1]:
import torch

x = torch.rand(10, 1, 28, 28)
print(x.shape)


torch.Size([10, 1, 28, 28])


In [2]:
print(x[0].shape)
print(x[1].shape)

torch.Size([1, 28, 28])
torch.Size([1, 28, 28])


In [3]:
print(x[0][0].shape)

torch.Size([28, 28])


In [4]:
import sys, os
from common.util import im2col
import numpy as np

x1 = torch.rand(1,3,7,7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = torch.rand(10,3,7,7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [8]:
import torch
import torch.nn as nn

class Convolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
    
    def forward(self, x):
        return self.conv(x)


In [9]:
import torch
import torch.nn as nn

class Pooling(nn.Module):
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        super().__init__()
        self.pool = nn.MaxPool2d(kernel_size=(pool_h, pool_w), stride=stride, padding=pad)
    
    def forward(self, x):
        return self.pool(x)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleConvNet(nn.Module):
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param={'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        super(SimpleConvNet, self).__init__()

        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]

        conv_output_size = (input_size - filter_size + 2 * filter_pad) // filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size // 2) * (conv_output_size // 2))

        self.conv1 = nn.Conv2d(in_channels=input_dim[0], out_channels=filter_num,
                               kernel_size=filter_size, stride=filter_stride, padding=filter_pad)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(pool_output_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

        self.conv1.weight.data.normal_(0, weight_init_std)
        self.fc1.weight.data.normal_(0, weight_init_std)
        self.fc2.weight.data.normal_(0, weight_init_std)

        self.conv1.bias.data.fill_(0)
        self.fc1.bias.data.fill_(0)
        self.fc2.bias.data.fill_(0)

    def forward(self, x):
        x = self.conv1(x) 
        x = F.relu(x) 
        x = self.pool(x)
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = F.relu(x) 
        x = self.fc2(x) 
        return x


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

model = SimpleConvNet(input_dim=(1, 28, 28), 
                      conv_param={'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1}, 
                      hidden_size=100, output_size=10, weight_init_std=0.01)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

max_epochs = 5
for epoch in range(max_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{max_epochs}], Loss: {running_loss / len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

model.eval() 
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')


Epoch [1/5], Loss: 2.2976, Accuracy: 11.82%
Epoch [2/5], Loss: 1.5062, Accuracy: 51.93%
Epoch [3/5], Loss: 0.3999, Accuracy: 88.08%
Epoch [4/5], Loss: 0.2988, Accuracy: 91.13%
Epoch [5/5], Loss: 0.2375, Accuracy: 92.94%
Test Accuracy: 94.13%
